In [31]:
import pandas as pd

high_schools = pd.read_pickle('pickle/high_schools.p')
high_schools.head()

,school_name,potential_college_students,latitude,longitude,start_time,end_time,subway_lines
dbn,,,,,,,
13K430,Brooklyn Technical High School,5323,40.688896,-73.976435,8:45am,3:15pm,"D, N to Atlantic Ave  Barclays Center; G to F..."
22K405,Midwood High School,3176,40.632829,-73.952356,8:45am,3:30pm,"2, 5 to Flatbush Ave  Brooklyn College"
10X445,Bronx High School of Science,2976,40.879958,-73.889011,8am,3:45pm,"4 to Bedford Park Blvd - Lehman College ; B, D..."
21K525,Edward R. Murrow High School,2754,40.619671,-73.959141,8:05am,2:45pm,Q to Ave M
03M485,Fiorello H. LaGuardia High School of Music & A...,2295,40.774202,-73.985976,8am,4pm,"1 to 66th St - Lincoln Center ; 2, A, B, C, D ..."


In [35]:
stations_location = pd.read_pickle('pickle/stations_location.p')
stations_location.head()

,station,latitude,longitude,lines,notes
0,Astor Pl,-73.991070,40.730054,4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
1,Canal St,-74.000193,40.718803,4-6-6 Express,"4 nights, 6-all times, 6 Express-weekdays AM s..."
2,50th St,-73.983849,40.761728,1-2,"1-all times, 2-nights"
3,Bergen St,-73.974999,40.680862,2-3-4,"4-nights, 3-all other times, 2-all times"
4,Pennsylvania Ave,-73.894886,40.664714,3-4,"4-nights, 3-all other times"


In [29]:
stations_traffic = pd.read_pickle('pickle/stations_traffic.p')
stations_traffic.head()

,station,avg_daily_traffic_feb
0,GRD CNTRL-42 ST,158625
1,42 ST-PORT AUTH,112739
2,34 ST-PENN STA,107514
3,TIMES SQ-42 ST,93595
4,34 ST-HERALD SQ,87825


In [23]:
# http://stackoverflow.com/questions/15736995/how-can-i-quickly-estimate-the-distance-between-two-latitude-longitude-points

from math import radians, cos, sin, asin, sqrt

def within_800m(school_lat, school_lon, station_lat, station_lon):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    school_lat, school_lon, station_lat, station_lon  = map(radians, 
                                                            [school_lat, school_lon, station_lat, station_lon])
    
    # haversine formula
    earth_radius_km = 6367
    d_lon = station_lon - school_lon 
    d_lat = station_lat - school_lat 
    a = sin(d_lat/2)**2 + cos(school_lat) * cos(station_lat) * sin(d_lon/2)**2
    c = 2 * asin(sqrt(a)) 
    m = earth_radius_km * 1000 * c
    return m <= 800

In [24]:
within_800m(40.688896, -73.976435, 40.6873877, -73.9748150)

True

In [25]:
within_800m(high_schools.ix[0, 'latitude'], high_schools.ix[0, 'longitude'], 
            stations_location.ix[0, 'latitude'], stations_location.ix[0, 'longitude'])

False